# PART 1: Basics of models, prompts & parsers

- **Openai_api_key:** set up using .env file
- **Templating prompts & outputs**
- **Parsing**: Extracting specific info from outputs
- **Formatting**: writing prompts to improve inferential and generative ability of LLM (ReAct: Thought, Action, Observation)

Import packages & specify basic completion function

In [ ]:
import openai
import os

# Connecting to account
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
# Uses GPT 3.5 Turbo
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}] 
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

Basic example use-case: translate and set tone

In [ ]:
message = """
Hey iedereen, met Karyan van Takumi. Morgen is het zover\
, we hebben er veel zin in! 📸Carmen, Rachel, Yip en Richie we zien elkaar 10:30 bij Takumi Nieuwstraat.\
Mochten er nog vragen zijn vandaag, stel ze gerust🙋🏻‍♀ Tot morgen
"""

In [ ]:
style = """ 
American English \
in a concise and confident tone, like a strict business executive
"""

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{message}```
"""

print(prompt)

In [ ]:
response = get_completion(prompt)
print(response)

## Langchain
Building a sequence of comptatible requests

In [ ]:
# pip install --upgrade Langchain

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# To control the randomness: temperature = 0.0
chat = ChatOpenAI(temperature=0)

print(chat)

## Prompt template
Useful for reusing long/useful prompts (with a layer of abstraction) with placeholder we can change along the way.

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

prompt_template.messages[0].prompt.input_variables

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
Hey bro, what the fuck is this a normal price, 
but you're employee charged me double the original price
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
customer_response = chat(customer_messages)

In [ ]:
customer_response.content

## Formatting
Chain-of-thought reasoning (ReAct):
- Thought: helps form better answers
- Action
- Observation: show what it learned from action 

Couple parser to extract interpret the output of these components.

In [ ]:
# Example output format (Dictionary / JSON)

{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [ ]:
# Input
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# Template to eventually get to JSON
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

How to wrap this in Langchain:

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0) # add api key as parameter ("openai_api_key") if not as .env parameter (recommended instead)
response = chat(messages) # Get response (note: output is type STRING)
print(response.content)

Output type here was STRING, now we want to parse to a dict/json

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Now we can get the output compressed in a dictionary object

In [ ]:
# Raw output
response = chat(messages)
# Output parser
output_dict = output_parser.parse(response.content)
type(output_dict)

---
# PART 2: Memory

- **ConversationBufferMemory:** store/extract messages as variables
- **ConversationBufferWindowMemory:** keep last *k* interactions as context/memory
- **ConversationTokenBufferMemory:** keep last interactions, capped by specified number of tokens as context/memory
- **ConversationSummaryMemory:** create summary of conversation as context/memory

additionally, Langchain also supports other memory types (beyond the scope of this notebook):
- **Vector Data Memory:** Stores text in vector database and retrieves most relevant blocks of text
- **Entity Memory:** Using LLM, remembers details about specific entities (objects)

Typically, if stored in databases, conversations are stored. They are stored as key-value pairs in SQL


In [ ]:
import openai
import os

# Connecting to account
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

import warnings
warnings.filterwarnings('ignore')

## ConversationBufferMemory (1/4)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
# Builds LLM
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose= True # With this you get insight in which prompts are ran under the hood (if True)
)

In [ ]:
conversation.predict(input = "Hi, I'm Richie")

conversation.predict(input = "do you remember my name?")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({}) # {} = empty dictionary, handy for more advanced use-cases


In [ ]:
memory = ConversationBufferMemory()

In [ ]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

## ConversationBufferWindowMemory (2/4)
LLM are stateless: i.e. each transaction is independent
Memory storage is used as additional **context** to have more conversation-like content.

A lot of context, directly implies **more tokens** and costs are charged by number of tokens consumed. Hence we should manage it well

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k = 1) # k = iteration it keeps (1 input + 1 output)

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

## ConversationTokenBufferMemory (3/4)
Memory limits nr of tokens saved to manage costs

- different LLM use different ways of counting tokens, hence we initialise a new one.

In [ ]:
!pip install tiktoken

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0) # different LLM use different ways of counting tokens, hence we initialise a new one.

In [ ]:
# Sample conversation
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100) # specify token limit here (priortises recent exchanges)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

## ConversationSummaryMemory (4/4)

Using LLM to write a summary of past conversation and using that as memory/context.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

# Insert few conversational terms
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [ ]:
# Summary of the conversation so far
memory.load_memory_variables({})

In [ ]:
# Verbose doesn't show actual OpenAI message, but still good.
Conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True,
)

In [ ]:
# Now make a request, including the memory
Conversation.predict(input = "what would be a nice demo to show?")

In [ ]:
# Note: latest message is now included
memory.load_memory_variables({})

Keep specific storage of messages up the limit of tokens. and uses CLM (casual language model?) to generate and save past data as summary

---
# PART 3: Chains

Different ways to compounding multiple prompts 
- **Simple Chain**
- **Sequential Chain:** multiple inputs / multiple outputs
- **Router Chain:** allocate traffic along various different chains

In [ ]:
import openai
import os

# Connecting to account
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

## LLMChain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature = 0.9)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Blenders with funny designs"
chain.run(product)

## Simple Sequential Chains
combine chains where output chain 1, is input chain 2

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run(product)

## SequentialChain

Multiple inputs and multiple outputs

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [ ]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = "Located in the Rotterdam Food Hall this wonderful Vietnamese stall serves excellent and authentic Vietnamese fare. Excellent service, reasonably priced and scrumptious food. Highly recommended"
overall_chain(review)

## Router Chain
Allocate inputs to sub-route chains, dependent on situations.

If no chain seems appropriate: **default chain**

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0)

In [ ]:
# Templates
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), # Important to help chain decide how to direct traffic amongst sub-chains
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

warning:
*C:\Users\richi\anaconda3\Lib\site-packages\langchain\chains\llm.py:279: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.*

In [ ]:
# Try questions
chain.run("What's your favorite character in naruto?")

---
## PART 4: Questions & Answers over documents
Working with embeddings & vector stores to handle big datasets / inputs

**Embedding methods:**
- **Stuff** (simple, not good for big data)
- **Map-reduce** (Summarise independent responses in single response, costs more calls, treats all chunks/documents as independent)
- **Refine** (Iteratively building up answers, will lead to long answers, slow due to dependency, same number of call as map-reduce)
- **Map-rerank** (Do all individual calls, rank and select highest as in softmax)

**Terminology**
- **Retriever**: interface to interact with documents (note: often the cause for poor outputs)

![](https://github.com/Richie-Lee/LLM_practice/blob/main/LangChain%20for%20LLM%20Application%20Development/embedding_llm.jpg)

In [ ]:
import openai
import os

# Connecting to account
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown # common display utilities

In [ ]:
filepath = "C:/Users/richi/Downloads/Cereals.csv"
cereal_loader = CSVLoader(file_path = filepath)

In [ ]:
from langchain.indexes import VectorstoreIndexCreator # helps create vector stoire

In [ ]:
# pip install docarray

In [ ]:
# Specify vector store class
index = VectorstoreIndexCreator(
    vectorstore_cls = DocArrayInMemorySearch, # easy one
).from_loaders([cereal_loader]) # specify all the docs here

In [ ]:
# To check whether the csv is not broken
import pandas as pd
file_path = 'C:/Users/richi/Downloads/Cereals.csv'
df = pd.read_csv(file_path) # Read the CSV file into a DataFrame
print(df.head()) 

In [ ]:
# Query over document (here csv) in loader
query ="Please list the average sodium value per mfr\
in a table in markdown"
response = index.query(query)

In [ ]:
# Using markdown visualisation package
display(Markdown(response))

**Embedding practice**

- Divide in chunks (Here: Docs / CSV rows) and allocate content to best content-matching chunk
- retrievers 

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
docs = cereal_loader.load() # each page = 1 CSV row

In [ ]:
# Example that shows what the embedding may look like
embed = embeddings.embed_query("Hi, I'm richie")
print(f"length: {len(embed)}, snippet: {embed[:5]}")

In [ ]:
# Create embeddings for all pieces of text, and save in vector store & "from-document method"
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [ ]:
# Based on the query it redirects to the embeddings with best matches (docs)
query = "Type C fruit"
docs = db.similarity_search(query)

In [ ]:
# Using this to Q&A - retriever is generic interface to fetch documents
retriever = db.as_retriever()

In [ ]:
# for NLP responses
llm = ChatOpenAI(temperature = 0) # Temperature controls randomness

Helpful ideas to scale:

In [ ]:
# Combine documents in single piece of text:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
fruit product in a table in markdown and summarize each one.") 

display(Markdown(response))

In [ ]:
# can be encapsulated in langchain chain,
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", # different processing methods (stuff = simplest)
    retriever=retriever, 
    verbose=True
)

In [ ]:
# Run chain on query
query =  "Please list all the products with a sodium level above 100 \
in markdown and summarize each one."

response = qa_stuff.run(query)
display(Markdown(response))

In [ ]:
display(Markdown(response))

In [ ]:
# Above was manual embedding creation, we can also do it more compactly as follows:
response = index.query(query, llm=llm)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

---
# PART 5: Evaluation 
*Prompting-based* development. Using LLM to extract semantic meanings ("new" evaluation heuristic)

- Wrong answer, often cause of poor retrieval, rather than inadequate model
- debugging for context interpretability & token usage (cost management)
- Manually/LLM-assisted example creation as labelled data



In [ ]:
import openai
import os

# Connecting to account
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
filepath = "C:/Users/richi/Downloads/Cereals.csv"
cereal_loader = CSVLoader(file_path = filepath)
data = cereal_loader.load()

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

**Example based evaluation: Manual**

Come up with good examples (Q&A pairs): 
- example input
- example ground truth (label)

In [ ]:
# Manual
examples = [
    {
        "query": "Are there products that contain apples?",
        "answer": "Yes, we have Apple Cinnamon Cheerios and Apple Jacks"
    },
    {
        "query": "What are the top 3 highest rated products?",
        "answer": "Here is a list with the top 5: 1. All-Bran with Extra Fiber, 2. Shredded Wheat nBran, 3. Shredded Wheat spoon size."
    }
]

In [ ]:
# Automate: 
from langchain.evaluation.qa import QAGenerateChain

**Example based evaluation: Automated/Assisted using LLMs**

In [ ]:
# Takes in document and creates question/answer pairs using a LLM
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

# UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.

In [ ]:
new_examples[0:4]

In [ ]:
# Add to existing examples
examples += new_examples

In [ ]:
# Shows inside the chain (limiting info though)
qa.run(examples[0]["query"])

**Debugging**:

In [ ]:
# Helps interpret what's happening under the hood - showing what's happening under the hood
import langchain
langchain.debug = True

qa.run(examples[0]["query"])

# Often when something goes wrong, it's not the model what's wrong but thr retrieval steps
# Tracks tokens

In [ ]:
# Turn off the debug mode
langchain.debug = False

LLM assisted evaluation


In [ ]:
# Predictions for all examples 
predictions = qa.apply(examples)

# Fails -> appears to struggle with generated examples

In [ ]:
# Create evaluation chain with language model
from langchain.evaluation.qa import QAEvalChain
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

**Example of how it should look:**

Example 0:
Question: Do the Cozy Comfort Pullover Set have side pockets?
Real Answer: Yes
Predicted Answer: The Cozy Comfort Pullover Set, Stripe does have side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: What is the weight of each pair of Women's Campside Oxfords?
Real Answer: The approximate weight of each pair of Women's Campside Oxfords is 1 lb. 1 oz.
Predicted Answer: The weight of each pair of Women's Campside Oxfords is approximately 1 lb. 1 oz.
Predicted Grade: CORRECT

Example 3:
Question: What are the dimensions of the small and medium Recycled Waterhog Dog Mat?
Real Answer: The dimensions of the small Recycled Waterhog Dog Mat are 18" x 28" and the dimensions of the medium Recycled Waterhog Dog Mat are 22.5" x 34.5".
Predicted Answer: The small Recycled Waterhog Dog Mat has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".
Predicted Grade: CORRECT


---
# PART 6: Agents (reasoning engines)

Build (debuggable) tools, that leverage chains that resemble logic. 

Examples covered:
- Calculator
- Wikipedia scraping (connecting wikipedia API)

Also covers how to customise tools/agents of your own
- Here: get today's date



In [ ]:
import openai
import os

# Connecting to account
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0) # Temperature = 0, to remove randomness in (carefully designed) reasoning

In [ ]:
# API running against wikipedia
# pip install -U wikipedia

**Calculator & Wikipedia search:**

In [ ]:
# Load tools (LLM-math = calculator, Wikipedia = API to search wikipedia)
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [ ]:

agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, # CHAT: optimised for chat_models, REACT: designed to get best reasoning
    handle_parsing_errors=True, # Useful when output is not parsable to action -> pass misformatted text back to LLM to correct itself
    verbose = True
)

In [ ]:
# Math test (good to understand what's happening the hood)
agent("What is the 25% of 300?")

In [ ]:
# Using Wikipedia API test
question = "I'm interested in learning about naruto, who is the main female character in this series?"
result = agent(question) 

**Python Agent:**

REPL = way to interact with code (roughly speaking like Jupyter notebook)

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

In [ ]:
# View detailed outputs of the chains
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

**Defining a personalised Tool**

Connect to own sources / API

In [ ]:
# pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
# Building a custom tool -> make sure to design the prompt such that the LLM will know when to direct traffic to this tool 
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \ 
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
# Note: when not reaching an answer - token usage can inflate if not bounded (max 4097 tokens)
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

agent.run("What's the date today?")

Note: Agents may reach wrong conclusions (currently still WIP). 

In [ ]:
# Rerun if wrong answer is reached
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")